In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("HDFS Test") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

df = spark.read.csv("hdfs://namenode:9000/edata.csv")
df.show()

+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------------+----------------+--------------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------+---------------+------------+
|                 _c0|               _c1|                 _c2|                 _c3|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|                 _c9|                _c10|                _c11|                _c12|             _c13|              _c14|            _c15|                _c16|            _c17|            _c18|                _c19|                _c20|                _c21|                _c22|  